In [5]:
import pandas as pd
from sodapy import Socrata
import datetime
import matplotlib.pyplot as plt
import numpy as np

In [6]:
domain = 'data.sfgov.org'
app_token = 'TQqk1kfcIaDLukBlSTtay4NNz'

client = Socrata("data.sfgov.org", app_token, timeout = 30)

# dictionaries by sodapy.
dataset = 'nuek-vuh3'
metadata = client.get_metadata(dataset)
columns = metadata.get('columns')
fieldNames = [col['fieldName'] for col in columns]

In [7]:
def getAll(domain, app_token, dataset,fieldList,limit):
    results = list()
    for i in range(limit):
        client = Socrata(domain, app_token, timeout = 30)
        incoming = client.get(dataset, select = fieldList)
        results.append(incoming)
       
        if len(incoming)<1000:
            break

        pctComplete = 100* (len(results)/limit)
        if pctComplete % 10==0:
            print('percent complete :',pctComplete,  ' %')
            
    return results

In [8]:
def jsonToDF(json):
    results_df = pd.DataFrame()
    for i in range(len(json)):
        results_df= results_df.append(pd.DataFrame.from_records(json[i]))
    return results_df

In [9]:
dataTypeDict = {'dispatch_dttm':'datetime64',
                'response_dttm':'datetime64',
                'call_Type':'str',
                'fire_prevention_district':'str',
                'neighborhoods_analysis_boundaries':'str',
                'number_of_alarms':'int',
                'original_priority':'str',
                'priority':'str',
                'unit_type':'str'}  

In [11]:
fieldList = 'dispatch_dttm, response_dttm, call_Type, fire_prevention_district, neighborhoods_analysis_boundaries, number_of_alarms, original_priority, priority, unit_type'

results = getAll(domain,app_token,dataset,fieldList,limit=10)
results_df = jsonToDF(results)        


results_df = results_df.astype(dataTypeDict)
results_df['response duration'] = results_df['response_dttm'] - results_df['dispatch_dttm']
results_df['response duration'] = results_df['response duration']/np.timedelta64(1,'s')/60
results_df.dtypes

percent complete : 10.0  %
percent complete : 20.0  %
percent complete : 30.0  %
percent complete : 40.0  %
percent complete : 50.0  %
percent complete : 60.0  %
percent complete : 70.0  %
percent complete : 80.0  %
percent complete : 90.0  %
percent complete : 100.0  %


dispatch_dttm                        datetime64[ns]
response_dttm                        datetime64[ns]
call_Type                                    object
fire_prevention_district                     object
neighborhoods_analysis_boundaries            object
number_of_alarms                              int32
original_priority                            object
priority                                     object
unit_type                                    object
response duration                           float64
dtype: object